In [18]:
%pylab inline
import h5py
import pyBigWig
from riboraptor .wig import WigReader
import pybedtools
from riboraptor.helpers import read_bed_as_intervaltree, yield_intervals

Populating the interactive namespace from numpy and matplotlib


# Problem
Given two bigwigs: one from the positive strand and one from the negative, we are interested in retaining only those pileups where the GTF gene and the corresponding pileup exist.
So for each gene, we will retain pileups from only one strand depending on the protocol.

If it is forward stranded:
 - For genes defined on the positive strand in the GTF: retain only values from the positive.bigwig
 - For genes defined on the negative strand in the GTF: retrain only values from the negative.bigwig
 

In [9]:
hdf = '/staging/as/skchoudh/re-ribo-analysis/hg38/SRP055009/hdf/SRX876723.hdf5'
pos_bw = '/staging/as/skchoudh/re-ribo-analysis/hg38/SRP055009/bigWig_lengthwise/SRX876723/28/5prime_pos.bw'
neg_bw = '/staging/as/skchoudh/re-ribo-analysis/hg38/SRP055009/bigWig_lengthwise/SRX876723/28/5prime_neg.bw'
out_bw = '/staging/as/skchoudh/SRX876723_28_5prime_collapsed.bw'
bed = '/home/cmb-panasas2/skchoudh/github_projects/riboraptor/riboraptor/annotation/hg38/gene.bed.gz'

In [16]:
bed_intervaltree = read_bed_as_intervaltree(bed)

In [10]:
hdf_read = h5py.File(hdf, 'r')

In [11]:
hdf_read.attrs['protocol']

'forward'

In [12]:
bed_df = pybedtools.BedTool(bed).sort().to_dataframe()
bed_df['chrom'] = bed_df['chrom'].astype(str)
bed_df['name'] = bed_df['name'].astype(str)
bed_grouped = bed_df.groupby('name')

if not isinstance(pos_bw, WigReader):
    pos_bw = WigReader(pos_bw)

if not isinstance(pos_bw, WigReader):
    neg_bw = WigReader(pos_bw)
    
chrom_sizes = pos_bw.chromosomes


In [14]:
out_bw = pyBigWig.open(out_bw, 'w')
out_bw.addHeader(list(chrom_sizes.items()), maxZooms=0)



In [24]:
bed_intervaltree['chr1'].find(10000, 100000)

['+', '-', '-', '+', '-', '+', '+', '+', '-', '-']

In [ ]:
for gene_name, gene_group in bed_grouped:
    assert len(gene_group['strand'].unique()) == 1
    gene_strand = gene_group['strand'].unique()[0]
    for index, row in gene_group.iterrows():
        chrom = row['chrom']
        start = row['start']
        end = row['end']
        strand = row['strand']
        if strand == '+':
            values = pos_bw.query(chrom, start, end)
        else:
            values = pos_bw.query(chrom, start, end)
            
        bw_sum += np.nansum(bw.values(chrom, start, end))
            